## optimization
1. 모델의 output과 label 간 loss 계산
2. loss의 derivative 수집
3. 경사하강법으로 paramter optimization

In [9]:
# pre-requisite 
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root='data',
    download=False,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=False,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__() #NeuralNetwork의 부모클래스인 nn.Module의 메서드, 속성을 사용할 수 있도록 참조하고 __init__으로 초기화
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),            
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()

In [10]:
# hyperparameter setting
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [11]:
# loss function
loss_fn = nn.CrossEntropyLoss() # initialize loss function 
 

In [12]:
# optimizer: paramter 조정
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [15]:
# train loop
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad() # parameter의 변화도 재설정(기본적으로 add up 되므로)
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() # 모델이 정확하게 예측한 데이터 수
    test_loss /= num_batches
    correct /= size # 정확도 계산
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [16]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.149421  [   64/60000]
loss: 2.144899  [ 6464/60000]
loss: 2.088249  [12864/60000]
loss: 2.102420  [19264/60000]
loss: 2.050022  [25664/60000]
loss: 1.982573  [32064/60000]
loss: 2.011881  [38464/60000]
loss: 1.932507  [44864/60000]
loss: 1.934571  [51264/60000]
loss: 1.854535  [57664/60000]
Test Error: 
 Accuracy: 58.4%, Avg loss: 1.855845 

Epoch 2
-------------------------------
loss: 1.882905  [   64/60000]
loss: 1.851765  [ 6464/60000]
loss: 1.740154  [12864/60000]
loss: 1.782098  [19264/60000]
loss: 1.672513  [25664/60000]
loss: 1.626092  [32064/60000]
loss: 1.648009  [38464/60000]
loss: 1.556381  [44864/60000]
loss: 1.579621  [51264/60000]
loss: 1.469934  [57664/60000]
Test Error: 
 Accuracy: 61.8%, Avg loss: 1.492579 

Epoch 3
-------------------------------
loss: 1.553760  [   64/60000]
loss: 1.519784  [ 6464/60000]
loss: 1.384125  [12864/60000]
loss: 1.455056  [19264/60000]
loss: 1.340660  [25664/60000]
loss: 1.337845  [32064/600